In [ ]:
%load_ext autoreload
%autoreload 2

import os
ipy = get_ipython()
from os.path import join as pjoin
import os,sys; sys.path.append(os.path.expandvars('$CODE_MEMORY_ERRORS'))
import error_sensitivity
import pandas as pd

data_dir_general = os.path.expandvars('$DATA_QUENTIN')
data_dir_input = os.path.expandvars('$DATA_MEMORY_ERRORS_STAB_AND_STOCH')
scripts_dir = pjoin( os.path.expandvars('$CODE_MEMORY_ERRORS'))

import gc
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
print('seaborn ver = ', sns.__version__)
#assert sns.__version__ == '0.12.2', sns.__version__
import matplotlib as mpl

from config2 import path_fig
path_fig_nf = pjoin(path_fig, 'NeuroFrance')

In [ ]:
def setPlotColors():
    COLOR = 'black'
    mpl.rcParams['text.color'] = COLOR
    mpl.rcParams['axes.labelcolor'] = COLOR
    mpl.rcParams['xtick.color'] = COLOR
    mpl.rcParams['ytick.color'] = COLOR


    sns.set(rc={'axes.facecolor':'white', 'figure.facecolor':'white',
               'grid.color':'grey', 'grid.alpha':0.5})
    global tgtc
    tgtc = ['b','g','r','brown']
setPlotColors()

In [ ]:
d = '/home/demitau/data_Quentin/pilot_studies/context_change_behav/'
#fnb = 'dima_test_context_change_20230309_164851'
#fnb = '2023-SE1-pilot2_context_change_20230310_161928'
fnb = '2023-SE1-017_context_change_20230425_161049'
fn = fnb + '.log'
fnp = fnb + '.param'

# Main

In [ ]:
import gc; gc.collect()

In [ ]:
from datetime import datetime
from behav_proc import readParamFiles, addBasicInfo, getGeomInfo
fnp = fnb + '.param'
params, phase2trigger, trigger2phase, stage2pars  = readParamFiles(fnp, d)
home_position, target_coords = getGeomInfo(params)
hitr = float(params['radius_target']) + float(params['radius_cursor']) / 2

load = True
#load = True
if load:
    #dfallphs = pd.read_pickle(pjoin(d,'row=frame_allph.pkl.zip'))
    dfc_all      = pd.read_pickle(pjoin(d,'row=frame.pkl.zip'))        
    dfcpc_all    = pd.read_pickle(pjoin(d,'row=pause.pkl.zip'))
    dfctc_all    = pd.read_pickle(pjoin(d,'row=target_chagne.pkl.zip'))
    fnf_dfcc = pjoin(d,'row=trial.pkl.zip')
    dfcc_all     = pd.read_pickle(fnf_dfcc)
    
    dt = datetime.fromtimestamp(os.stat(fnf_dfcc).st_mtime )
    print('date dfcc = ',dt)
        
dfcc_all['error_lh_ang_deg'] = dfcc_all['error_lh_ang'] * 180 / np.pi
dfcc_all['error_lh2_ang_deg'] = dfcc_all['error_lh2_ang'] * 180 / np.pi
dfcc_all['error_lh2_ang_deg_abs'] = dfcc_all['error_lh2_ang_deg'].abs()

assert dfcc_all.loc[dfcc_all['subject'].str.contains('pilot'), 'subject'].nunique() == 1
dfcc_all['subject_ind'] = dfcc_all['subject'].str[-3:]
dfcc_all.loc[dfcc_all['subject'].str.contains('pilot'), 'subject_ind'] = 0
dfcc_all['subject_ind'] = dfcc_all['subject_ind'].astype(int)
#.astype(int)

subjects = dfcc_all['subject'].unique()
print(subjects)

subj = subjects[3]
print(subj)
dfccos  = dfcc_all.query('subject == @subj')
dfcos   = dfc_all.query('subject == @subj')
dfcpcos = dfcpc_all.query('subject == @subj')
#dfcc = dfccos

#dfcc_all['error_lh_ang_deg'] = dfcc_all['error_lh_ang'] * 180 / np.pi
#dfcc_all['error_lh_ang_deg_abs'] = dfcc_all['error_lh_ang_deg'].abs()
#dfcc_all['error_area2_signed_nn_abs'] = dfcc_all['error_area2_signed_nn'].abs()
dfcc_all['target_locs_nice'] = np.round(dfcc_all['target_locs']).astype('int') - 90

In [ ]:
#dfcc_all['error_unpert_area2_signed_nn']

In [ ]:
# when exited home and not spec trial
from behav_proc import spectrials, coln2pubname
nclean_trials = dfcc_all.query('~time_lh.isna() and ~trial_type.isin(@spectrials)').\
  groupby(['subject','block_ind']).size().min()
print('min clean trials in a block ', nclean_trials)      

In [ ]:
ncn = 4
if load:
    fnf = pjoin(d,f'es_row_trial_ncn{ncn}.pkl.zip')
#fnf = pjoin(d,'es_row=trial.pkl.zip')
    dt = datetime.fromtimestamp(os.stat(fnf).st_mtime )    
    print('date es = ',dt)
    df_all_multi_tsz_orig = pd.read_pickle(fnf)

df_all_multi_tsz_orig['target_locs_nice'] = np.round(df_all_multi_tsz_orig['target_locs']).astype('int') - 90

In [ ]:
df_all_multi_tsz_orig['error_lh2_ang_deg'] = \
    df_all_multi_tsz_orig['error_lh2_ang'] * 180 / np.pi

c = df_all_multi_tsz_orig['error_data_for_calc'] == 'lh2_ang'
df_all_multi_tsz_orig['error_plot'] = df_all_multi_tsz_orig['error']
df_all_multi_tsz_orig.loc[c,'error_plot'] = df_all_multi_tsz_orig.loc[c,'error_lh2_ang_deg']

In [ ]:
df_all_multi_tsz_orig['error_coln_for_plot'] =  df_all_multi_tsz_orig['error_coln_for_calc']
for k,v in coln2pubname.items():
    df_all_multi_tsz_orig['error_coln_for_plot'] = \
        df_all_multi_tsz_orig['error_coln_for_plot'].str.replace(k, v)

In [ ]:
for df_ in [dfcc_all    ,dfc_all     ,dfcpc_all   ,dfctc_all]:
    assert np.sum( df_.duplicated()) == 0
    
assert dfcc_all.index.duplicated().sum() == 0

In [ ]:
df_all_multi_tsz = df_all_multi_tsz_orig

In [ ]:
mx = df_all_multi_tsz.groupby(['subject','trial_index']).size().max()
print('Duplication for ES = ',mx)

In [ ]:
# info about ES dataset
cols_undup = ['subject', 'trial_index', 'error_data_for_calc',
       'trial_shift_size', 'trial_group_col_calc', 'time_locked']
assert not df_all_multi_tsz.duplicated(cols_undup).any()
for col in cols_undup:
    if col == 'trial_index':
        continue
    r = df_all_multi_tsz[col].unique()
    if col == 'subject':
        r = len(r)
    print(col, r)

## Process err sens

In [ ]:
[s for s in df_all_multi_tsz.columns if 'nonhit' in s ]

In [ ]:
gc.collect()

In [ ]:
#df_all_multi_tsz = pd.read_pickle(pjoin(d,'es_row=trial.pkl.zip'))

coln_pairs = [  ('error_lh2_ang_valid','error_unpert_lh2_ang_valid',
                'error_lh2_ang_valid_nonhit','lh2_ang_valid'),
                ('error_area2_signed_nn_valid', None,
               'error_area2_signed_nn_valid_nonhit',
                'signed_area2_nn_valid'),
              ('error_lh2_ang','error_unpert_lh2_ang',
                'error_lh2_ang_nonhit','lh2_ang'),
                ('error_area2_signed_nn', None,
               'error_area2_signed_nn_nonhit',
                'signed_area2_nn') ]

coln = 'err_sens'
calc_names = df_all_multi_tsz['error_data_for_calc'].unique()
assert set( calc_names ) == set(  [cp[-1] for cp in coln_pairs])

In [ ]:
df_all_multi_tsz['inv_prev_error'] = 1. / df_all_multi_tsz['prev_error']

In [ ]:
numtrain = params['num_training']

In [ ]:
from behav_proc import qs_motor_var
coln = 'is_motor_var_est_trial'
df_all_multi_tsz[coln] = False
df_all_multi_tsz.loc[df_all_multi_tsz.query(qs_motor_var).index,
                     coln] = True

df_all_multi_tsz['prev_' + coln] = \
    df_all_multi_tsz.groupby(['subject'] + cols_undup[2:])\
    [coln].shift(1)

In [ ]:
cols_undup

In [ ]:
df_all_multi_tsz['prev_error_area2_signed_nn'] = \
    df_all_multi_tsz.groupby(['subject'] + cols_undup[2:])['error_area2_signed_nn'].shift(1)

In [ ]:
from behav_proc import truncateDf
#dfc_multi_tsz = df_all_multi_tsz.loc[inds1.append(inds2)]
dfc_multi_tsz = truncateDf(df_all_multi_tsz, 
        'err_sens', q=None, infnan_handling='discard',
        cols_uniqify = cols_undup[2:])

## Truncation

In [ ]:
0.07 * 180 / np.pi

In [ ]:
coln2mvar_std

In [ ]:
from behav_proc import qs_not_easy0, qs_notspec, spectrials
from behav_proc import qs_motor_var, printRejectInfo
print(qs_motor_var)
numtrain = params['num_training']
dfcc_motor_var = dfcc_all.query(qs_motor_var)
print(len(dfcc_motor_var))
coln2mvar_std = {}
for cp in coln_pairs:
    coln = cp[0]
    #print(coln)
    desc = dfcc_motor_var[coln].abs().describe()
    display( desc )
    std = desc.to_dict()['std']
    coln2mvar_std[coln] = std
    
    if '_ang' in coln:
        print( 'std = {:.4f}, in deg = {:.2f}'.
              format(std, std * 180 / np.pi) )
    else:
        print( f'std = {std:.4f}')
    printRejectInfo(dfcc_all, coln, std)
    printRejectInfo(dfcc_all, coln, std/2)

In [ ]:
from behav_proc import qs_motor_var, printRejectInfo
tl = 'target'
qs_es_ = ('error_data_for_calc == @edfc_'
          ' and trial_shift_size == 1 '
          ' and trial_group_col_calc == "trials"'
          ' and time_locked == @tl and trial_index >= @numtrain')

#dfcc_motor_var = dfcc_all.query(qs_motor_var)
coln2mvar_std_preverr = {}
coln2mvar_std_corr = {}
for cp in coln_pairs:
    coln = cp[0]
    edfc_ = cp[-1]
    
    dfes_ = df_all_multi_tsz.query(qs_es_)
    dfes_motor_var = dfes_.query(
        'prev_is_motor_var_est_trial == True and ' 
        'is_motor_var_est_trial == True')
    print(len(dfes_motor_var))

    print(coln)
    desc = dfes_motor_var['prev_error'].abs().describe()
    display( desc )
    std = desc.to_dict()['std']
    coln2mvar_std_preverr[coln] = std
    
    desc2 = dfes_motor_var['correction'].abs().describe()
    display( desc2 )
    std2 = desc2.to_dict()['std']
    coln2mvar_std_corr[coln] = std2
    
    if '_ang' in coln:
        print( 'std = {:.4f}, in deg = {:.2f}'.
              format(std, std * 180 / np.pi) )
    else:
        print( f'std = {std:.4f}')
    printRejectInfo(dfes_, coln, std)
    printRejectInfo(dfes_, coln, std/2)
    
    

### make plots

In [ ]:
from behav_proc import row2multierr, plotTraj3
from behav_proc import plotAllTajs, qs_easy1, qs_easy2, qs_easy_wide
        
plotAllTajs(dfcc_all.query(qs_easy_wide).iloc[:300], dfc_all, params,
           traj_to_plot = ['org_feedback'])
ax=plt.gca()
ax.legend([])

In [ ]:
#df_ = dfcc_motor_var.query('time_tmstart >= 0.55')
df_ = dfcc_motor_var.query('time_tmstart >= 0.58')
#df_ = dfcc_motor_var.query('time_tmstart >= 0.62')
plotAllTajs(df_,
            dfc_all,  params, exitpt_col = 'time_tmstart2',
           verbose=1)
ax=plt.gca()
ax.legend([])

In [ ]:
len(dfccos) * .2

In [ ]:
dfccos['subject']

In [ ]:
df_ = dfccos.query('error_lh2_ang_deg_abs > 0.4').\
    sample(frac = 0.25)

In [ ]:
pjoin(path)

In [ ]:
    #nlargest(150,'error_lh2_ang_deg_abs')
plotAllTajs(df_,
            dfc_all,  params, exitpt_col = 'time_tmstart2',
           verbose=0 , traj_to_plot = ['org_feedback'])
ax=plt.gca()
ax.legend([])
ax.set_title(f'{len(df_)} UNperturbed trajectories for subject {subj[-3:]}')

plt.savefig(pjoin( path_fig_nf, 'org_feedback.pdf') )

In [ ]:
plt.figure()
plotAllTajs(df_,
            dfc_all,  params, exitpt_col = 'time_tmstart2',
           verbose=0 , traj_to_plot = ['feedback'])
ax=plt.gca()
ax.legend([])
ax.set_title(f'{len(df_)} perturbed trajectories for subject {subj[-3:]}')

plt.savefig(pjoin( path_fig_nf, 'feedback.pdf') )

In [ ]:
#for bi in [4,5]:
# blue is true
for bi in [2, 3,4]:
    df_ = dfccos.query('block_ind == @bi')

    plt.figure()
    plotAllTajs(df_.iloc[:5],
                dfc_all,  params, exitpt_col = 'time_tmstart2',
               verbose=0 , traj_to_plot = ['org_feedback','feedback'])
    ax=plt.gca()
    ax.legend([])
    pert = df_['perturbation'].values[0]
    ax.set_title(f'Block {bi} (N={len(df_)}) block perturbation = {pert}')
    ax.set_xlim(-250,250)
    plt.savefig(pjoin( path_fig_nf, f'org_and_feedback_bi={bi}.pdf') )

In [ ]:
df

In [ ]:
df_[['is_easy_wide','time_lh','time_tmstart_jax1','time_tmstart_jax2', 'time_tmstart2','ang_fb_lh','ang_lh_to_tgt','error_lh2_ang_deg','error_lh_ang_deg']]

In [ ]:
df_['time_tmstart2']

In [ ]:
df_ = dfcc_all.query('error_lh2_ang_deg_abs <= 1').\
    nlargest(70,'error_lh2_ang_deg_abs')
plotAllTajs(df_,
            dfc_all,  params, exitpt_col = 'time_tmstart2',
           verbose=1 , traj_to_plot = ['org_feedback'])
ax=plt.gca()
ax.legend([])

In [ ]:
from behav_proc import calcAdvErrors

In [ ]:
df_ = dfcc_all.nlargest(18,'error_lh_ang_deg')
plotAllTajs(df_,
            dfc_all, params,verbose=1, exitpt_col = 'time_lh')
ax=plt.gca()
ax.legend([])

In [ ]:
df_ = dfcc_motor_var.nlargest(50,'time_tmstart')
plotAllTajs(df_,
            dfc_all, params,verbose=1, exitpt_col = 'time_tmstart2')
ax=plt.gca()
ax.legend([])

In [ ]:
df_ = dfcc_motor_var.nsmallest(20,'error_lh2_ang_deg')
plotAllTajs(df_,
            dfc_all, params,verbose=1, exitpt_col = 'time_tmstart2')
ax=plt.gca()
ax.legend([])

In [ ]:
df_ = dfcc_motor_var.nsmallest(50,'error_lh2_ang_deg_abs')
plotAllTajs(df_,
            dfc_all, params, verbose=1, exitpt_col = 'time_tmstart2')
ax=plt.gca()
ax.legend([])

In [ ]:
df_ = dfcc_motor_var.nlargest(20,'error_lh2_ang_deg')
plotAllTajs(df_,
            dfc_all, params, verbose=1, exitpt_col = 'time_tmstart2')
ax=plt.gca()
ax.legend([])

In [ ]:
df_ = dfcc_motor_var.nlargest(20,'error_lh_ang_deg')
plotAllTajs(df_,
            dfc_all, verbose=1)
ax=plt.gca()
ax.legend([])

In [ ]:
1000 / 120 * 5

In [ ]:
df_ = dfcc_all.nlargest(50,'error_lh_ang_deg')
plotAllTajs(df_,
            dfc_all, verbose=1)
ax=plt.gca()
ax.legend([])

In [ ]:
df_ = dfcc_all.nsmallest(50,'error_lh_ang_deg')
plotAllTajs(df_,
            dfc_all, verbose=1)
ax=plt.gca()
ax.legend([])

In [ ]:
dfcc_all['error_lh2_ang_deg_abs'] = dfcc_all['error_lh2_ang_deg'].abs()

In [ ]:
df_ = dfcc_all.nsmallest(30,'error_lh2_ang_deg_abs')
plotAllTajs(df_,
            dfc_all, params, verbose=1)
ax=plt.gca()
ax.legend([])

In [ ]:
#dftr.plot?

In [ ]:
dftmp = dfcc_motor_var.nlargest(10,'error_lh_ang_deg').copy()

In [ ]:
df = pd.DataFrame({
    'group': ['A', 'A', 'B', 'B', 'B'],
    'data': [1, 2, 3, 4, 5]
})

# group by 'group' column and compute rolling standard deviation
df['rolling_std'] = df.groupby('group')['data'].expanding().std().reset_index(drop=True)

In [ ]:
# def f(dftmp):
#     return dftmp['jax1'].expanding().std()
# dfc_all['jax1_std'] = dfc_all.groupby(['subject','trial_index']).apply(f)
    
    



In [ ]:
dftr[['jax1_std','jax2_std']].values.shape

In [ ]:
from behav_proc import addTrueMvtTime
_ = addTrueMvtTime(dfcc_all, dfc_all)
gc.collect()

# trunc stats

In [ ]:
descr = dfcc_motor_var.groupby('subject')['time_tmstart'].describe()
descr = descr.rename(columns=dict(zip(descr.columns, [c+'_ot' for c in descr.columns])) )
print('Mean over subject of means over trials')
descr.describe().drop('count')

In [ ]:
#dfcc_motor_var['ttms1m2'] = (dfcc_motor_var[]'time_tmstart_jax1'] - dfcc_motor_var_['time_tmstart_jax2'])
descr = dfcc_motor_var.groupby('subject')['ttms1m2'].describe()
descr = descr.rename(columns=dict(zip(descr.columns, [c+'_ot' for c in descr.columns])) )
print('Mean over subject of means over trials')
descr.describe().drop('count')

In [ ]:
descr = dfcc_motor_var.groupby('subject')['time_tmstart'].describe()
descr = descr.rename(columns=dict(zip(descr.columns, [c+'_ot' for c in descr.columns])) )
print('Mean over subject of means over trials')
descr.describe().drop('count')

In [ ]:
descr = dfcc_motor_var.groupby('subject')['error_lh2_ang_deg'].describe()
descr = descr.rename(columns=dict(zip(descr.columns, [c+'_ot' for c in descr.columns])) )
print('Mean over subject of means over trials')
dede = descr.describe().drop('count')
dede

In [ ]:
descr = dfcc_motor_var.groupby('subject')['error_lh2_ang_deg_abs'].describe()
descr = descr.rename(columns=dict(zip(descr.columns, [c+'_ot' for c in descr.columns])) )
print('Mean over subject of means over trials')
dedeabs = descr.describe().drop('count')
display( dedeabs )

mestd = dedeabs.to_dict()['std_ot']['mean']
n = (dfcc_all['error_lh2_ang_deg'].abs() < mestd).sum()
prop = n / len(dfcc_all)
print(f'Mean of stds = {mestd:.2f}, prop = {prop * 100:.1f}%')

In [ ]:
descr = dfcc_motor_var.query('error_lh2_ang_deg_abs < 1').groupby('subject')['error_lh2_ang_deg_abs'].describe()
descr = descr.rename(columns=dict(zip(descr.columns, [c+'_ot' for c in descr.columns])) )
print('Mean over subject of means over trials')
dedeabs_sm = descr.describe().drop('count')
dedeabs_sm

In [ ]:
descr = dfcc_motor_var.groupby('subject')['error_area2_signed_nn'].describe()
descr = descr.rename(columns=dict(zip(descr.columns, [c+'_ot' for c in descr.columns])) )
print('Mean over subject of means over trials')
dedeabs = descr.describe().drop('count')
display( dedeabs )

mestd = dedeabs.to_dict()['std_ot']['mean']
n = (dfcc_all['error_area2_signed_nn'].abs() < mestd).sum()
prop = n / len(dfcc_all)
print(f'Mean of stds = {mestd:.2f}, prop = {prop * 100:.1f}%')

In [ ]:
from behav_proc import printRejectInfo
for mestd_ in np.linspace(1e-5, 4, 13):
    printRejectInfo(dfcc_all, 'error_lh2_ang_deg', mestd_)
    printRejectInfo(dfcc_motor_var, 'error_lh2_ang_deg', mestd_)

In [ ]:
for mestd_ in np.linspace(1e-5, 2.2, 18):
    printRejectInfo(dfcc_all, 'error_area2_signed_nn', mestd_)
    #printRejectInfo(dfcc_motor_var, 'error_area2_signed_nn', mestd_)

In [ ]:
thr_ang  = 1.
thr_area = 0.4

In [ ]:
calc_names = df_all_multi_tsz['error_data_for_calc'].unique()

In [ ]:
# choose variation for not easy context?

In [ ]:
len( dfcc_motor_var.query('error_lh_ang_deg.abs() > 10') ) / len(dfcc_motor_var)

In [ ]:
dfcc_motor_var['error_lh_ang_deg'].idxmax()

In [ ]:
subjects

In [ ]:
dfcc_motor_var['error_lh_ang_deg'].describe()

In [ ]:
zip(['signed_area2_nn', 'lh_ang', 'signed_area2_nn_scaled_ed'],
   [None, 2, None ])

In [ ]:
printRejectInfo(dfcc_all, 'error_lh2_ang_deg', thr_ang)

In [ ]:
dfes = df_all_multi_tsz.query('error_data_for_calc == "lh_ang"')
from behav_proc import thr_lh_ang_rad as thr_ang
inds1 = dfes.query('(prev_error.abs() <= @thr_ang)' ).index

dfes = df_all_multi_tsz.query('error_data_for_calc == "signed_area2_nn_scaled_ed"')
from behav_proc import thr_signed_area2_nn_scaled_ed as thr_area
inds2 = dfes.query('(prev_error >= @thr_area) or (prev_error <= -@thr_area)' ).index

In [ ]:
len(df_all_multi_tsz)

## pairplots

In [ ]:
dfcc_motor_var.hist(column=['error_lh2_ang_deg','error_lh_ang_deg'],
                    bins=30, sharex=0)

In [ ]:
sns.pairplot(data=dfcc_motor_var_, x_vars=['time_lh'],
            y_vars=['ttms1m2'], hue='subject')

In [ ]:
dfcc_all['tmstart_after_lh'] = dfcc_all['time_tmstart'] - dfcc_all['time_lh']

In [ ]:
dfcc_all['tmstart_after_lh']

In [ ]:
dfcc_all.groupby('subject')['tmstart_after_lh'].mean().nlargest(5)

In [ ]:
sns.pairplot(data=dfcc_all, vars=['subject','tmstart_after_lh'], corner=True )

In [ ]:
sns.pairplot(data=dfcc_all, vars=['subject_ind','time_lh','time_tmstart',
            'ttms1m2','tmstart_after_lh','error_lh_ang_deg','error_lh2_ang_deg'], corner=True )

In [ ]:
sns.pairplot(data=dfcc_motor_var, x_vars=['subject','time_lh','time_tmstart_jax1','time_tmstart_jax2','error_lh_ang_deg'],
            y_vars=['ttms1m2','time_lh','time_tmstart_jax1','time_tmstart_jax2','ttms1m2'])

In [ ]:
100 * len(dfc_multi_tsz) / len(df_all_multi_tsz)

In [ ]:
gc.collect()

### basic plots

In [ ]:
for cn in calc_names:
    dfc_ = dfc_multi_tsz.query('trial_shift_size == 1 and error_data_for_calc == @cn')
    plt.figure(figsize=(12,3))
    #dfc_ = df_all_multi_tsz.query('trial_shift_size == 1')
    ax = sns.lineplot(x="trials", y='err_sens',data=dfc_,  estimator='mean', 
                      errorbar='se',
                     hue = 'trial_group_col_calc')#, hue='block')
    ax.axhline(y=0, c='red', ls=':')
    ax.legend(loc='lower right')
    ax.set_title(cn)

In [ ]:
dfone = df_all.query('subject == @subjects_[0]')
yname = coln_pairs[0][0]

ebarg = ''
ax = sns.lineplot(x="trials", y=yname,data=dfone,  estimator='mean', errorbar='se')#, hue='block')
ax.axhline(0,ls=':',c='red')
ax.plot(dfone['trials'].to_numpy(), dfone['perturbation'].to_numpy(), ls=':',c='y')


##### violin plots

In [ ]:
fg = sns.catplot(kind='violin', data=dfcc_all,
                 y='error_area2_signed_nn_scaled_ed',                                   
                 x='perturbation', hue='trial_type', 
                 col = 'special_block_type',
                 figsize=(4,3))
for ax in fg.axes.flatten():
    #ax.set_ylim(-1e5,1e5)
    ax.axhline(0,c='r',ls=':')

In [ ]:
fg = sns.catplot(kind='violin', data=dfcc_all,
                 y='error_lh_ang',                                   
                 x='perturbation', hue='trial_type', 
                 col = 'special_block_type',
                 figsize=(4,3))
for ax in fg.axes.flatten():
    ax.set_ylim(-100,100)
    ax.axhline(0,c='r',ls=':')

In [ ]:
#dfc_.groupby(['subject', 'trial_index']).size()

In [ ]:
#for cn in calc_names:
for df_to_plot in [dfc_multi_tsz]: #, df_all_multi_tsz]:
    dfc_ = df_to_plot.query('trial_shift_size == 1 and trial_group_col_calc == "trials" and '
                            ' trial_type != "error_clamp" and '
                           ' special_block_type != "error_clamp_sandwich"')  
    # error_data_for_calc = @cn
    assert dfc_.groupby(['subject', 'trial_index']).size().max() == 3
    print(len(dfc_))
    #plt.figure(figsize=(6,3))
    #dfc_ = df_all_multi_tsz.query('trial_shift_size == 1')
    fg = sns.catplot(kind='violin', data=dfc_,
                     y='err_sens', col = 'error_data_for_calc',                                   
                     x='perturbation', figsize=(6,3))
    #x = 'dist_rad_from_prevtgt2',
    for ax in fg.axes.flatten():
        ax.set_ylim(-7,7)
        ax.axhline(0,c='r',ls=':')
        #ax.set_title(cn)

In [ ]:
#for cn in calc_names:
for df_to_plot in [dfc_multi_tsz]: #, df_all_multi_tsz]:
    dfc_ = df_to_plot.query('trial_shift_size == 1 and trial_group_col_calc == "trials" and '
                            ' trial_type != "error_clamp" and '
                           ' special_block_type != "error_clamp_sandwich"')  
    # error_data_for_calc = @cn
    assert dfc_.groupby(['subject', 'trial_index']).size().max() == 3
    print(len(dfc_))
    #plt.figure(figsize=(6,3))
    #dfc_ = df_all_multi_tsz.query('trial_shift_size == 1')
    fg = sns.catplot(kind='violin', data=dfc_,
                     y='err_sens', col = 'error_data_for_calc',                                   
                     x='perturbation', row='subject', figsize=(6,3))
    #x = 'dist_rad_from_prevtgt2',
    for ax in fg.axes.flatten():
        ax.set_ylim(-7,7)
        ax.axhline(0,c='r',ls=':')
        #ax.set_title(cn)

In [ ]:
#for cn in calc_names:
for df_to_plot in [dfc_multi_tsz]: #, df_all_multi_tsz]:
    dfc_ = df_to_plot.query('trial_shift_size == 1 and trial_group_col_calc == "trials" and '
                            ' trial_type != "error_clamp" and '
                           ' special_block_type != "error_clamp_sandwich"')  
    # error_data_for_calc = @cn
    assert dfc_.groupby(['subject', 'trial_index']).size().max() == 3
    print(len(dfc_))
    #plt.figure(figsize=(6,3))
    #dfc_ = df_all_multi_tsz.query('trial_shift_size == 1')
    fg = sns.catplot(kind='violin', data=dfc_,
                     y='err_sens', col = 'error_data_for_calc',                                   
                     x='perturbation', row='tgti_to_show', figsize=(6,3))
    #x = 'dist_rad_from_prevtgt2',
    for ax in fg.axes.flatten():
        ax.set_ylim(-7,7)
        ax.axhline(0,c='r',ls=':')
        #ax.set_title(cn)

In [ ]:
# all subj separately
subj = subjects[0]
df_to_plot = df_all_multi_tsz

ynames_add = ['error_endpoint_ang', 'jax2']
ynames_every = ['err_sens', 'correction']

titest = 18
ww = 12; hh = 2
nr = len(calc_names) * (len(ynames_every) + 2) + len(ynames_add);
nc = 1
fig, axs = plt.subplots(nr,nc, figsize=(nc*ww,nr*hh))
axs = axs.reshape((nr,nc))
def fplot(ax, dfc_, yn):
    ax = sns.lineplot(x="trials", y=yn, data=dfc_,  estimator='mean', 
                      errorbar='se',hue = 'trial_group_col_calc', ax=ax)#, hue='block')
    ax.axhline(y=0, c='red', ls=':')
    ax.legend(loc='lower right')

axi = 0
for cni, cn in enumerate(calc_names):
    #df_all_multi_tsz
    #df_to_plot = dfc_multi_tsz    
    dfc_ = df_to_plot.query('trial_shift_size == 1 and trial_group_col_calc == "trials" '
            'and error_data_for_calc == @cn and trial_index < @titest and subject == @subj')        
    for yn in ynames_every:
        ax = axs[axi, 0 ]
        fplot(ax, dfc_, yn)
        ax.set_ylabel(f'{yn}: {cn}')
        ax.set_title(f'{yn}: {cn}')
        axi += 1
        
    for ii in range(2):
        ax = axs[axi, 0 ]
        yn = coln_pairs[cni][ii]
        fplot(ax, dfc_, yn)
        ax.set_title(f'{yn}: {cn}')
        axi += 1
        
    
for yn in ynames_add:
    ax = axs[axi, 0]
    fplot(ax, dfc_, yn)
    axi += 1

for ax in axs.flatten():
    ax.set_xlim(0,titest)
    #dfc_ = df_all_multi_tsz.query('trial_shift_size == 1')
    
plt.tight_layout()


In [ ]:
import gc; gc.collect()

##### REST and  move prep anomalise

In [ ]:
tis_cur = tis[10:30]
tis_cur = tis[10:11]
dfr = dfc_all.query('subject == @subjects[0] and trial_index in @tis_cur')

In [ ]:
dfr = dfr.sort_values('time')

In [ ]:
dfr.shift

In [ ]:
dfccos['time_mvt'] = -100

In [ ]:
#dfccos = dfccos.copy()

In [ ]:
inds

In [ ]:
dfccos.query()

In [ ]:
#dfccos_hit = dfccos[~dfccos['nonhit'] ]
#for ti in tis:


In [ ]:
dfccos['time_mvt']

In [ ]:
#nfs.describe()

In [ ]:
nf1  = float( params['time_at_home'] ) * float(params['FPS'] )
nf2 = float( params['motor_prep_duration'] ) * float(params['FPS'] )
print(nf1,nf2)

In [ ]:
triggs

In [ ]:
nf1 * 1.5

In [ ]:
subj2nt = df_unproc[['subject','trial_index']].groupby('subject').nunique()
subj2nt = subj2nt.to_dict('index')
#{'2023-SE1-001': {'trial_index': 637},
# '2023-SE1-002': {'trial_index': 636},

In [ ]:
subj2nt

In [ ]:
subj2qv

In [ ]:
subj2qv

In [ ]:
for phase in ['REST', 'GO_CUE_WAIT_AND_SHOW']:
    print(phase)
    phases = [phase]
    triggs = [phase2trigger[ph] for ph in phases]
    df0 = df_unproc.query('current_phase_trigger in @triggs')
    
    nfs = df0.groupby(['subject','trial_index']).size().\
        reset_index().rename(columns={0:'numframes'})
    
    
    dftmp = nfs.groupby('subject')['numframes'].quantile(0.75).reset_index().set_index('subject')
    subj2qv = dftmp.to_dict('index')
    print(subj2qv)
    
    def f(row):
        qv = subj2qv[row['subject']]['numframes']
        return row['numframes'] > qv
    nfs['bad'] = nfs.apply(f,1)
    nfs
    
    display( nfs.describe() )
    
    #qv = nfs['numframes'].quantile(0.5)
    #qv = nf1 * 1.5
    nfs_bad = nfs[nfs['bad']]
    
    grp = nfs_bad[['subject','trial_index']].groupby('subject')
    display(grp.size())
    
    plt.figure()
    sns.lineplot(nfs, x='trial_index', y ='numframes')

In [ ]:
for phase in ['REST', 'GO_CUE_WAIT_AND_SHOW'][1:]:
    print(phase)
    phases = [phase]
    triggs = [phase2trigger[ph] for ph in phases]
    df0 = df_unproc.query('current_phase_trigger in @triggs')
    
    nfs = df0.groupby(['subject','trial_index']).size().\
        reset_index().rename(columns={0:'numframes'})
    sns.lineplot(nfs.query('subject == @subjects[0]'), x='trial_index', y ='numframes')

In [ ]:
#nfs_bad.query('sub')


##### plot time resolved with pert and blocks

In [ ]:
(dfc_multi_tsz['trial_type'] == 'error_clamp').sum()

In [ ]:
from config2 import path_fig

In [ ]:
#subj = subjects[1]
dfr_all = []
for subj in subjects:

    #df_to_plot = df_all_multi_tsz
    df_to_plot = dfc_multi_tsz

    #' and special_block_type != "error_clamp_sandwich"'
    qs0 = ('trial_shift_size == 1 and trial_group_col_calc == "trials"'
           ' and trial_type != "error_clamp0"'                      
                f' and subject == "{subj}"')
    #dfc0_ = df_to_plot.query(qs0) 
    r = []
    for cni, cn in enumerate(calc_names):
        #df_all_multi_tsz
        #df_to_plot = dfc_multi_tsz            
        qs = qs0 + f' and error_data_for_calc == "{cn}" '
        dfc_ = df_to_plot.query(qs)

        print(len(dfc_))

        dfc_ = dfc_.set_index('trials')
        if cni != 0:
            dfc_ = dfc_[['err_sens','correction']]
        dfc_ = dfc_.rename(columns={'err_sens': f'err_sens:{cn}', 'correction': f'correction:{cn}'})
        r += [ dfc_]

    #[dfc0_.drop(labels=['err_sens','correction'], axis=1)]
    dfr0 = pd.concat( r, axis=1)
    print(len(dfr0))

    dfr0 = dfr0.reset_index()
    
    print( 'num nans = ', dfr0['trial_index'].isna().sum() )
    dfr = dfr0[~dfr0['trial_index'].isna() ] 

    dfr_all += [dfr]
    #dfr['block_ind']
    #dfr.query('trial_type != "error_clamp"').groupby('block_ind')
    #coln_pairs

    ynames = []
    for yn1, yn2, nh, cn in coln_pairs:
        ynames += [f'err_sens:{cn}', f'correction:{cn}', yn1, yn2]
    ynames += ['error_endpoint_ang']
    print(ynames)

    #assert len(dfc_) == len(dfr), (len(dfc_), len(dfr))
    #dfr[['trial_index','err_sens:signed_area','err_sens:lh_ang']]
    dfcpcos_, dfcos_ = dfcpc_all.query('subject == @subj'), dfc_all.query('subject == @subj')

    assert 0 == sum( dfr['trial_index'].duplicated() )

    xlim = 0,150
    # dfc_ = dfr.query('trial_shift_size == 1 and trial_group_col_calc == "trials" '
    #         'and error_data_for_calc == @cn and trial_index < @xlim[1] and subject == @subj')
    dfr_ = dfr.query('trial_index < @xlim[1]')

    from behav_proc import multiplot
    #['err_sens', 'correction']
    multiplot(dfr_, dfcpcos_, dfcos_, ynames, start_ind=0, xlim = xlim , ww=14, hh=1.5,
             show_titles = True)
    plt.tight_layout()
    #plt.savefig( pjoin(path_fig, f'{subj}_{",".join(ynames)}.pdf' ) )
    plt.savefig( pjoin(path_fig, f'{subj}_nr={len(ynames)}_xlim={xlim}.pdf' ) )
    plt.close()
    
dfr_all = pd.concat(dfr_all).reset_index()

In [ ]:
dfr['trial_index'].isna().sum()

In [ ]:
print( dfr['trial_index'].isna().sum() )
print( dfr_all['trial_index'].isna().sum() )
dfr_all = dfr_all[~dfr_all['trial_index'].isna() ] 

In [ ]:
#dfr_all['subject']

In [ ]:
subjects = list(sorted(subjects))

In [ ]:
subjects_subset

In [ ]:
sns.lineplot(dfcc, x='trial_index', y ='feedback_abs')

In [ ]:
sns.lineplot(dfcc, x='trial_index', y ='feedback')

In [ ]:
dfcc['error_intdist']

In [ ]:
dfcc.columns

# error plots

In [ ]:
# TODO maybe I should set error to nan for pause trials

In [ ]:
#pause_inds = dfcpcos['trial_index'].to_numpy()
pause_inds = dfcpcos['trial_index'].unique()
block_ends = dfcos.groupby(['block_ind'])['trial_index'].max()

In [ ]:
dfccos  = dfcc_all.query('subject == @subj').copy()

In [ ]:
dfccos['error_lh_ang_deg'] = dfccos['error_lh_ang'] * 180 / np.pi

In [ ]:
numtrain = params['num_training']

In [ ]:
from nf_poster_plots import *
locs = oneSubjErrDyn(dfccos, pause_inds, subj=subj)
#axs= locs['axs']
plt.close()

# summary one remnants

In [ ]:
pause_inds = np.array(pause_inds)
dfccos['pause_rel'] = -100
dfccos.loc[dfccos['trial_index'].isin(pause_inds-1), 'pause_rel'] = -1
dfccos.loc[dfccos['trial_index'].isin(pause_inds+1), 'pause_rel'] = 1
# for pi in pause_inds:
#     piprev=  pi -1
#     pinext=  pi +1
#     df_ = dfccos.query('trial_index == @piprev')


In [ ]:
#df_['pause_rel']

In [ ]:
num_coupled_EC = \
    np.sum( dfccos.query("trial_type == 'error_clamp'")['trial_index'].diff() == 1 )
print(f'num_coupled_EC = {num_coupled_EC}')

In [ ]:
len(block_starts), len(block_ends)

In [ ]:


block_starts = grp['trial_index'].min()
block_starts = block_starts.to_dict()

block_ends = grp['trial_index'].max()
block_ends = block_ends.to_dict()


block_bounds = {}
for bi in block_starts.keys():
    block_bounds[bi] = block_starts[bi],block_ends[bi]
block_bounds

## Start of block vs end of block

In [ ]:
dfccos.columns

In [ ]:
dfccos['trial_type'].unique()

In [ ]:
grp = dfccos.query('trial_type != "error_clamp"').groupby('block_ind')

df1 = grp['trial_index'].min().to_frame().rename(columns={'trial_index':'block_start_trial_index'})
df2 = grp['trial_index'].max().to_frame().rename(columns={'trial_index':'block_end_trial_index'})
block_bounds = pd.concat( [df1, df2], axis=1)

sts = block_bounds.query('block_ind >= 0')['block_start_trial_index']._values
df_block_start = dfccos.loc[dfccos['trial_index'].isin(sts)]

ends = block_bounds.query('block_ind >= 0')['block_end_trial_index']._values
df_block_end = dfccos.loc[dfccos['trial_index'].isin(ends)]

ynames = ['error_pert_adj', 'error_area_signed']
#'error', 


df_block_start = df_block_start[['block_ind', 'trial_type'] + ynames].set_index('block_ind')
#ynames_mod = [yn + '_start' for yn in ynames]
#rd = dict(zip(ynames,ynames_mod))
#df_block_start = df_block_start.rename(columns=rd)
df_block_start['type'] = 'start'

df_block_end =   df_block_end[['block_ind', 'trial_type'] + ynames].set_index('block_ind')
df_block_end['type'] = 'end'

df_startend = pd.concat([df_block_start, df_block_end])

In [ ]:
df_startend

In [ ]:
for yname in ynames:
    plt.figure()
    sns.violinplot(df_startend, y=yname, x='type', hue='trial_type', split=True)

In [ ]:
df_ = dfccos.query('pause_rel > -10')
#df_ = df_.query('error < 150')

ynames = ['error_area_signed',  'error_distance', 'error']
for yname in ynames:
    plt.figure()
    
#     if yname  == 'error':
#         df_ = df_.query('error < 150')
    sns.violinplot(df_, y=yname, x='pause_rel')

In [ ]:
df_.index

# summary many

In [ ]:
# dfcc_all_ = dfcc_all.drop(labels=['level_0', 'index'], axis=1)
# dfcc_all_ = dfcc_all_.reset_index().drop(labels=['index'], axis=1)
dfcc_all_ = dfcc_all

In [ ]:
dfcc_all['target_locs_nice']

In [ ]:
numtrain = int(params['num_training'])
from behav_proc import qs_notspec_not_afterspec, spectrials
qs = qs_notspec_not_afterspec
df_ = dfcc_all.query(qs)

### Reaction time

In [ ]:
#same plot legend group by var

In [ ]:
#dfcc.columns

In [ ]:
df_all_multi_tsz['dist_rad_from_prevtgt2']

In [ ]:
[col for col in df_all_multi_tsz.columns if col.find('calc') >= 0]

In [ ]:
df_['dist_rad_from_prevtgt'].max()

In [ ]:
df__

In [ ]:
df_all_multi_tsz['early'] = df_all_multi_tsz['block_ind'] < max_bi / 2

In [ ]:
df_all_multi_tsz['early'].unique()

In [ ]:
gc.collect()

In [ ]:
df__ = df_all_multi_tsz.query(qs + ' and error_data_for_calc == @edfc')  
# there is much more point for dist_rad_from_prevtgt2 == 0
fg = sns.catplot(df__, kind='violin', y='time_lh', col='dist_rad_from_prevtgt2', row='early')
for ax in fg.axes.flatten():
    ax.set_ylim(-0.1,0.8)
    ax.axhline(ls=':',c='r')

In [ ]:
#dfes = dfcc.query(qs)# and err_sens >= -20 and err_sens <= 20')
fg = sns.relplot(df_,kind='line', y='time_lh', x='trial_index')#, x = 'error_data_for_calc')
# for ax in fg.axes.flatten():
#     ax.set_ylim(-10,10)

In [ ]:
7/4

In [ ]:
fg = sns.relplot(df_,kind='line', y='time_lh', x='trialwb')#, x = 'error_data_for_calc')

In [ ]:
df_.groupby(['subject', 'tgti_to_show', 'vis_feedback_type', 'trialwb']).size()

In [ ]:
ynames

In [ ]:
dfc_multi_tsz['error_data_for_calc'].unique()

In [ ]:
dfc_multi_tsz['trial_shift_size'].unique()

In [ ]:
len(dfes), len(dfc_multi_tsz)

In [ ]:
grp = dfes.groupby(['subject','error_data_for_calc'])
display(grp.size() )
grp = dfes.groupby(['error_data_for_calc'])
display(grp['err_sens'].mean())

In [ ]:
dfes = dfc_multi_tsz.query(qs + ' and trial_shift_size == 1')# and err_sens >= -20 and err_sens <= 20')
fg = sns.catplot(dfes, kind='violin',y='err_sens', x = 'error_data_for_calc')
for ax in fg.axes.flatten():
    ax.set_ylim(-3,3)
    ax.axhline(ls=':',c='r')

# Violins

In [ ]:
coln_pairs

In [ ]:
dfext_ = dfcc_all_.query('trial_index > @numtrain and trial_type != "error_clamp"')
fg = sns.catplot(dfext_, kind='violin', x='prev_trial_type', 
                 y='error_area2_signed_nn_scaled_ed', col = 'special_block_type')

fg = sns.catplot(dfext_, kind='violin', x='prev_trial_type', 
                 y='error_lh_ang', col = 'special_block_type')


# dfext_ = dfc_multi_tsz.query('trial_index > @numtrain and trial_type != "error_clamp"')
# fg = sns.catplot(dfext_, kind='violin', x='prev_trial_type', 
#                  y='error_area_signed_nn_scaled_ed', col = 'special_block_type')

# 3x3

In [ ]:
colnes = ['error_lh_ang_deg', 'error_area2_signed_nn_scaled_ed']
colnes = ['error_lh_ang_deg', 'error_area2_signed_nn']
colnes_abs = ['error_lh_ang_deg_abs', 'error_area2_signed_nn_abs']

In [ ]:
sns.relplot?

In [ ]:
sns.lineplot?

In [ ]:
fg = sns.lineplot(df_, kind='line', x='trialwb', y='error_lh_ang', 
                 col='vis_feedback_type', 
            row='tgti_to_show',errorbar='se')
for ax in fg.axes.flatten():
    ax.axhline(0,ls=':',color='red')

In [ ]:
mpl.rcParams.keys()

In [ ]:
#mpl.rcParams['ytick.labelsize'] = 15
#sns.set(rc={'axes.facecolor':'white', 'figure.facecolor':'white',
#           'grid.color':'grey', 'grid.alpha':0.5})

In [ ]:
calc_names

In [ ]:
calc_names_err_plot = ['lh2_ang', 'signed_area2_nn']

In [ ]:
# sns.set(rc={'axes.facecolor':'white', 
#             'figure.facecolor':'white'})


In [ ]:
sns.relplot?

In [ ]:
#sns.set(font_scale=1.5)
sns.set_style("whitegrid", {'font_scale':2.9})

In [ ]:
df_ = df_all_multi_tsz.query('error_data_for_calc.isin(@calc_names_err_plot)  and error_data_for_calc == "signed_area2_nn"')
df_= df_.rename(columns= {'target_locs_nice':'Target angle'})

df_= df_.rename(columns= {'error_coln_for_plot':'Error type'})

In [ ]:
# I'll remove last trial 10 because it is rarely present
# and creates too much variability
from nf_poster_plots import pubnames
#fd = {'fontsize':15}
#dfttl = {'fontsize':14}
#colne = 'error_lh_ang'
errorbar = 'se'
fg = sns.relplot(data=df_.query('trialwb < 10'), 
                 kind='line', x='trialwb', y='error_plot', 
                 col='perturbation', hue='Error type',
            row='Target angle')#, errorbar=errorbar)
for ax in fg.axes.flatten():
    ax.axhline(0,ls=':',color='red')
    ax.set_xlabel('Trial within block'    )
    #ax.set_ylabel(pubnames['colne'], 
    #              fontdict=fd)
    ax.set_ylabel('Error')
    ttl = ax.title.get_text().replace('target_locs',
                        pubnames['target_locs'])
    ax.set_title(ttl)

plt.savefig( pjoin(path_fig_nf, 
    f'AllSubj_{errorbar}_decay_3x3_area_only.svg' ) )
plt.savefig( pjoin(path_fig_nf, 
        f'AllSubj_{errorbar}_decay_3x3_area_only.pdf' ) )
# plt.savefig( pjoin(path_fig_nf, 
#     f'AllSubj_{errorbar}_decay_3x3.svg' ) )
# plt.savefig( pjoin(path_fig_nf, 
#         f'AllSubj_{errorbar}_decay_3x3.pdf' ) )

In [ ]:
# I'll remove last trial 10 because it is rarely present
# and creates too much variability
from nf_poster_plots import pubnames
fd = {'fontsize':15}
dfttl = {'fontsize':14}
for colne in colnes:
    #colne = 'error_lh_ang'
    errorbar = 'se'
    fg = sns.relplot(df_.query('trialwb < 10'), kind='line', x='trialwb', y=colne, 
                     col='perturbation', 
                row='target_locs_nice', errorbar=errorbar)
    for ax in fg.axes.flatten():
        ax.axhline(0,ls=':',color='red')
        ax.set_xlabel('Trial within block',
                     fontdict=fd)
        ax.set_ylabel(pubnames[colne], 
                      fontdict=fd)
        ttl = ax.title.get_text().replace('target_locs',
                            pubnames['target_locs'])
        ax.set_title(ttl, fontdict=dfttl)

    plt.savefig( pjoin(path_fig_nf, 
            f'AllSubj_{colne}_{errorbar}_decay_3x3.pdf' ) )

In [ ]:
for colne in colnes:
    fg = sns.relplot(df_.query('perturbation != 0 and trialwb < 10'), 
                     kind='line', x='trialwb', 
                     hue='perturbation',
                     y=colne)
    for ax in fg.axes.flatten():
        ax.axhline(0,ls=':',color='red')
    #break

In [ ]:
fg = sns.relplot(df_, kind='line', x='trialwb',
                 y=colne, 
                 col='perturbation', 
            row='tgti_to_show', hue='Nctx_app', height=3)
for ax in fg.axes.flatten():
    ax.axhline(0,ls=':',color='red')

In [ ]:
from behav_proc import genCtxPairLists,setContextSimilarityCols
all_ctx, cpls = genCtxPairLists(dfcc_all)
setContextSimilarityCols(dfcc_all, cpls)       

# drops

In [ ]:
(~dfcc_all['prev_ctx_some_close'].isnull()).sum()

In [ ]:
(~dfcc_all['prev_block_ind_diff'].isnull()).sum()

In [ ]:
colnes = [cp[0] for cp in coln_pairs][2:]; colnes

In [ ]:
from behav_proc import calcErrorDrops
dropinds = [0,1,2,3,4]
dfdrop_incv, dfdrop2_incv, dfdrop_last_incv = calcErrorDrops(dfcc_all, colnes, 
                                     inc_veridical=1, inds = dropinds)

dfdrop, dfdrop2, dfdrop_last = calcErrorDrops(dfcc_all, colnes, 
                                     inc_veridical=0, inds = dropinds)

assert dfdrop.groupby(['subject','Nctx_app','coln','ctxid']).size().max() == 1
assert dfdrop2.groupby(['subject','droptoind','Nctx_app','coln','ctxid']).size().max() == 1
assert dfdrop.groupby(['subject','Nctx_app','coln']).size().max() == 6 
assert dfdrop_incv.groupby(['subject','Nctx_app','coln']).size().max()  == 9

In [ ]:
# for i,ind in enumerate(inds[1:] ):
#     dfdrop[f'absdrop_{ind}'] = dfdrop['absdrop'].apply(lambda x: x[i] if x is not None else None)

In [ ]:
dfdrop.columns

In [ ]:
dfneg = dfdrop2.query('absdrop < 0')
dfbad = dfneg

In [ ]:
def f(row):
    bi = row['block_ind']
    subj = row['subject']
    droptoind = row['droptoind']
    dfr = dfcc_all.query('subject == @subj and block_ind == @bi and trialwb == @droptoind')
    assert len(dfr) == 1, len(dfr)
    return dfr.iloc[0]['trial_index']
dfdrop2['trial_index'] = dfdrop2.apply(f,1)

In [ ]:
dfbad.columns

In [ ]:
dfdrop2.duplicated(['subject','droptoind','Nctx_app']).any()

In [ ]:
(~df_notspec_notver['prev_ctx_pert_same'].isnull()).sum()

In [ ]:
dfdrop.columns

In [ ]:
# r = dfdrop.groupby(['coln', 'subject','ctxid']).agg({'Nctx_app': ['min', 'max'], 
#                                                 'absdrop_4': lambda x: x.max() - x.min()})


In [ ]:
#sns.set(font_scale=1.5)
sns.set_style("whitegrid", {'font_scale':1.3})

### Absdrop 4

In [ ]:
for dfdrop_cur in [dfdrop_incv]:#, dfdrop]:
    df_ = dfdrop.query('coln == "error_area2_signed_nn"')
    fg = sns.catplot(data=df_ , kind='violin', y='absdrop_4',  
                 height=5, col='coln', x='Nctx_app', sharey = None)
    for axi,ax in enumerate(fg.axes.flatten()):
        ax.axhline(0,ls=':',color='red')
    #     if axi <= 9:
    #         ax.set_ylim(-0.5,1.2)
    #     else:        
    #         ax.set_ylim(-15,25)
    #     if axi < 1:
    #         ax.set_ylim(-0.5,1.2)
    #     else:        
    #         ax.set_ylim(-15,25)


        ax.set_ylim(-15,25)    

        ax.set_xlabel('N context appearance')
        ax.set_ylabel('Error drop after 4 trials')

        ax.set_title( coln2pubname[ colnes[1-axi] ]  )


    plt.savefig(pjoin(path_fig_nf, 'drop_vs_Nctx_app.svg'))
    plt.savefig(pjoin(path_fig_nf, 'drop_vs_Nctx_app.pdf'))

    fg = sns.catplot(data=df_, kind='violin', x='ctxid', y='absdrop_4',  
                 height=5, col='coln', sharey = "col")
    for axi,ax in enumerate(fg.axes.flatten()):
        ax.axhline(0,ls=':',color='red')
    #     if axi < 1:
    #         ax.set_ylim(-0.5,1.2)
    #     else:        
    #         ax.set_ylim(-15,25)

        ax.set_ylim(-15,25)
        ax.set_xlabel('Contex ID')
        ax.set_ylabel('Error drop after 4 trials')

        ax.set_title( coln2pubname[ colnes[1-axi] ]  )

    #plt.savefig(pjoin(path_fig_nf, 'drop_vs_ctxid.svg'))
    #plt.savefig(pjoin(path_fig_nf, 'drop_vs_ctxid.pdf'))

    plt.savefig(pjoin(path_fig_nf, 'drop_vs_ctxid_only_area.svg'))
    plt.savefig(pjoin(path_fig_nf, 'drop_vs_ctxid_only_area.pdf'))

    # fg = sns.catplot(dfdrop, kind='violin', y='absdrop_4',  
    #              height=4, row='coln', col='subject')
    # for ax in fg.axes.flatten():
    #     ax.axhline(0,ls=':',color='red')

### Absdrop less than 4

In [ ]:
len(df_)

In [ ]:
%matplotlib inline

In [ ]:
colnes

In [ ]:
c = dfdrop2['coln'] == "error_lh2_ang"
dfdrop2.loc[c, 'absdrop'] *= 180 / np.pi

In [ ]:
#for coln_drop in ['absdrop_1', 'absdrop_2', 'absdrop_4']:
for dfdrop_cur in [dfdrop2]:#, dfdrop]:        
#or droptoind in [1,2,3,4]:    #
    #df_ = dfdrop_cur.query('coln == "error_area2_signed_nn"')
    df_ = dfdrop_cur.query(f'absdrop.abs() <= 130')
    fg = sns.catplot(data=df_ , kind='box', y=coln_drop,  
                 height=5, col='coln', x='Nctx_app', sharey = None,
                     row = 'droptoind')
    axs = fg.axes.flatten()
    for axi,ax in enumerate(axs):
        ax.axhline(0,ls=':',color='red')

        ax.set_xlabel('N context appearance')
        ax.set_ylabel(f'Error drop after {coln_drop[-1]} trials')
        #ax.set_title( coln2pubname[ colnes[axi] ]  )
    #axs[1].set_ylim(-25,20)    
    #axs[0].set_ylim(-20,45)    

#     plt.savefig(pjoin(path_fig_nf, 'drop_vs_Nctx_app.svg'))
#     plt.savefig(pjoin(path_fig_nf, 'drop_vs_Nctx_app.pdf'))

#     fg = sns.catplot(data=df_, kind='violin', x='ctxid', y=coln_drop,  
#                  height=5, col='coln', sharey = "col")
#     for axi,ax in enumerate(fg.axes.flatten()):
#         ax.axhline(0,ls=':',color='red')
#         ax.set_ylim(-15,25)
#         ax.set_xlabel('Contex ID')
#         ax.set_ylabel(f'Error drop after {coln_drop[:-1]} trials')

#         ax.set_title( coln2pubname[ colnes[1-axi] ]  )

#     plt.savefig(pjoin(path_fig_nf, 'drop_vs_ctxid_only_area.svg'))
#     plt.savefig(pjoin(path_fig_nf, 'drop_vs_ctxid_only_area.pdf'))

In [ ]:
dfdrop.groupby('subject')

In [ ]:
from pingouin import ttest
ttr = ttest(dfdrop['absdrop_4'], 0, alternative='greater')
display(ttr)

In [ ]:
for coln in colnes:
    print(coln)
    print( len( dfdrop.query('coln == @coln and absdrop_4 > 1') ) )

### Absdrop average

In [ ]:
grpda = dfdrop2.groupby(['subject','Nctx_app','coln','ctxid'])
display( grpda.size().describe() )
dropav = grpda['absdrop'].mean()

In [ ]:
dropav = dropav.reset_index()

In [ ]:
coln_drop  = 'absdrop'
for dfdrop_cur in [dropav]:#, dfdrop]:
    #df_ = dfdrop_cur.query('coln == "error_area2_signed_nn"')
    df_ = dfdrop_cur.query(f'{coln_drop}.abs() <= 130' )
    fg = sns.catplot(data=df_ , kind='box', y=coln_drop,  
                 height=5, col='coln', x='Nctx_app', sharey = None)
    axs = fg.axes.flatten()
    for axi,ax in enumerate(axs):
        ax.axhline(0,ls=':',color='red')

        ax.set_xlabel('N context appearance')
        ax.set_ylabel(f'Error drop after {coln_drop[-1]} trials')
        ax.set_title( coln2pubname[ colnes[axi] ]  )
    #axs[1].set_ylim(-25,20)    
    #axs[0].set_ylim(-20,45)    

#     plt.savefig(pjoin(path_fig_nf, 'drop_vs_Nctx_app.svg'))
#     plt.savefig(pjoin(path_fig_nf, 'drop_vs_Nctx_app.pdf'))

#     fg = sns.catplot(data=df_, kind='violin', x='ctxid', y=coln_drop,  
#                  height=5, col='coln', sharey = "col")
#     for axi,ax in enumerate(fg.axes.flatten()):
#         ax.axhline(0,ls=':',color='red')
#         ax.set_ylim(-15,25)
#         ax.set_xlabel('Contex ID')
#         ax.set_ylabel(f'Error drop after {coln_drop[:-1]} trials')

#         ax.set_title( coln2pubname[ colnes[1-axi] ]  )

#     plt.savefig(pjoin(path_fig_nf, 'drop_vs_ctxid_only_area.svg'))
#     plt.savefig(pjoin(path_fig_nf, 'drop_vs_ctxid_only_area.pdf'))

### Absdrop stats

In [ ]:
for coln_drop in ['absdrop_1', 'absdrop_2', 'absdrop_4']:
    print(f'--------  {coln_drop} -------')
    for coln in colnes:
        print(coln)

        dfdrop_ = dfdrop.query('coln == @coln')
        app1 = dfdrop_.query('Nctx_app == 1')
        app2 = dfdrop_.query('Nctx_app == 7')

        from pingouin import ttest
        ttrs = []
        for alt in ['greater','less','two-sided']:
            ttr = ttest(app1[coln_drop], app2[coln_drop], alternative=alt)
            ttrs += [ttr]
        display(pd.concat(ttrs) )

        pvs = []
        ttrs = []
        for subj in subjects:
            for alt in ['greater','less','two-sided']:
                app1os = app1.query('subject == @subj')
                app2os = app2.query('subject == @subj')
                ttr = ttest(app1os[coln_drop], app2os[coln_drop] ,alternative=alt)
                ttr['subject'] = subj
                ttrs += [ttr]
                pv = ttr['p-val'].values[0]
                pvs += [pv]
            #display(ttr)
        ttrs = pd.concat(ttrs)
        d = ttrs['p-val'].describe()
        print(d['mean'], d['std'])
        #print('mean alt pv per subj' np.mean( pvs ) )

In [ ]:
for coln in colnes:
    fg = sns.relplot(dfdrop.query('coln == @coln'), 
            kind='line', x='Nctx_app', y='absdrop_4',  
            height=3, errorbar='se')
    for ax in fg.axes.flatten():
        ax.set_title(coln)
    #    ax.axhline(0,ls=':',color='red')

In [ ]:
dfdrop_ =dfdrop
fg = sns.relplot(dfdrop_, kind='line', x='Nctx_app', y='absdrop_4',  hue='ctxid',
            col='coln', height=3, errorbar='se')
for ax in fg.axes.flatten():
    ax.axhline(0,ls=':',color='red')
    #ax.set_ylabel('')
fg = sns.relplot(dfdrop_, kind='line', x='Nctx_app', y='absdrop_2',  hue='ctxid',
            col='coln', height=3, errorbar='se')
for ax in fg.axes.flatten():
    ax.axhline(0,ls=':',color='red')
    #ax.set_ylabel('')

In [ ]:
np.random.randint(0,3, size=20)

In [ ]:
df_notspec_notver = dfcc_all.query(qs_notspec + ' and vis_feedback_type != "veridical"'
                                  ' and trialwb <= 4 and Nctx_app in [1,7]')
for coln in ['error_area2_signed_nn_scaled_ed', 'error_lh_ang']:
    fg = sns.relplot(df_notspec_notver, kind='line', x='trialwb', y=coln, 
                     col='vis_feedback_type', hue='Nctx_app', 
                     row = 'subject', height=3, errorbar='se')
    for ax in fg.axes.flatten():
        ax.axhline(0,ls=':',color='red')
        ax.set_ylabel('')
        #ax.legend(loc='lower right')
    fg.fig.suptitle(coln)
    fg.fig.tight_layout()
    break

In [ ]:
df_os = df_.query('subject == @subjects[0] and vis_feedback_type == "rot20"')
fg = sns.relplot(df_os, kind='line', x='trialwb', y='error_area2_signed_nn_scaled_ed', 
                 col='vis_feedback_type', 
            row='tgti_to_show', hue='Nctx_app', height=3)
for ax in fg.axes.flatten():
    ax.axhline(0,ls=':',color='red')
    ax.set_ylabel('')

#### naive (don't use it)

In [ ]:


qs_es_switches = qs_es + ' and prev_block_ind_diff > 0'
#dfes = df_all_multi_tsz_orig.query(qs_es)
#NOTE: this would discard when first trial after switch failed
dfes_sw = dfc_multi_tsz.query(qs_es_switches).copy()

In [ ]:
df_all_multi_tsz.query(qs_es_switches).groupby('subject').size().describe()

In [ ]:
dfsw =df_all_multi_tsz.query(qs_es_switches)
grp = dfsw.groupby(['subject','block_ind'])
lbd = lambda x: x.loc[x['trial_index'].idxmin(), 'time_lh']
ti = grp.apply(lbd)

In [ ]:
=df_all_multi_tsz.query(qs_es_switches)

In [ ]:
qs_es_switches2 = qs_es + ' and trialwb == 0 and block_ind >= 0'
dfsw2 =df_all_multi_tsz.query(qs_es_switches2)
display(dfsw2.groupby('subject').size().describe())

# choosing right trials to take ES from

In [ ]:
qs_notspec

In [ ]:
edfc

In [ ]:
#cp = coln_pairs[1]
cp = coln_pairs[0]

coln_error = cp[0]
edfc = cp[-1]
print(cp, coln_error, edfc)
tl = 'target'
#tl = 'trial_end'
#f'subject == "{subj}"' 

In [ ]:
coln_pairs

In [ ]:
#dfcc_all.columns

In [ ]:
from behav_proc import extractCtxChangeRelevantRows
import traceback
dfes_sws = []
for cp in coln_pairs[:2]:
    edfc = cp[-1]
    print('Starting ', edfc)
    
    try:
        dfes_sw_ = extractCtxChangeRelevantRows(df_all_multi_tsz, 
                                       edfc, tl)
    except Exception as e:
        #raise e
        print('EXC',e)
        exc_info = sys.exc_info()
        exc = traceback.TracebackException(*exc_info, capture_locals=True)
        ei = exc_info[2]    
        display(ei.tb_frame)
        psf = ei

        lfprev = None; lf = None
        lfs = []
        di = 0
        while psf.tb_frame is not None:
            stackframe = psf.tb_frame
            display(di, stackframe)

            psf = psf.tb_next  
            if 'miniconda' not in stackframe.f_code.co_filename:        
                lfprev = lf
                lf = stackframe.f_locals
                lfs += [lf]
                di += 1
            if psf is None:
                break    
    
    dfes_sws += [dfes_sw_]
dfes_sw = pd.concat(dfes_sws)

In [ ]:
dfes_sw.duplicated().any()

#### dbg

In [ ]:
dfsw_care = lf['dfsw_care']

In [ ]:
cols

In [ ]:
s

In [ ]:
from behav_proc import plotAllTajs
plotAllTajs(df_.iloc[:3], dfc_all, params)

In [ ]:
edfc

In [ ]:
s = subjects[1]; print(s)
#56,58,59
df_ = df_all_multi_tsz.query('subject == @s and'
                      ' error_data_for_calc == @edfc'
                      ' and trial_index in [79,80,81]')
df_[cols + ['prev_time_lh', 'trial_group_col_calc','error_data_for_calc']]

In [ ]:
edfc = cp[-1]; edfc

In [ ]:
cols = ['trial_index','trialwb','trial_type','prev_trial_type','prev_error',
    'err_sens','error','time_lh']
dfsw_care.loc[dfsw_care['err_sens'].isna(),
    cols]

In [ ]:
df_all_multi_tsz['prev_time_lh']

In [ ]:
easy,prevnan = lf['easy'], lf['prevnan']

In [ ]:
(easy | prevnan).all()

In [ ]:
dfes_sw = dfes_sws[0]

In [ ]:
edfc = coln_pairs[0][-1]

# Spatial and pert generalizations



In [ ]:
dfcpcos.groupby('trial_index').min('time')

In [ ]:
gc.collect()

In [ ]:
coln_error

In [ ]:
drs = dfes_sw['dist_rad_from_prevtgt2'].unique();
drs_f = np.array( list(map( float, drs) ) )
drs = list(sorted(drs,key = lambda x: float(x)))
#drs = list(sorted(drs,key = lambda x: float(x)))
#drs = np.array(drs, dtype=int)
print(drs)
#dfc_s = dfc[dfc['trial_group_col_calc'] == 'trialwtgt_we']
# TODO: should I restrict to one env?
dfc_s = dfes_sw.query('trial_group_col_calc == "trialwe"')
grp = dfc_s.groupby(['trial_shift_size', 'dist_rad_from_prevtgt2'])
#dts = dfc_multi_tsz['trial_shift_size'].unique()

dps = dfes_sw['perturbation_diff_abs'].unique()
dps = np.array([dp for dp in dps if not np.isnan(dp)], dtype=int)
print(dps)
npe = len(dps)

In [ ]:
from behav_proc import plotPolys, getPvals

In [ ]:
drpairs = [(drs[i],drs[i+1] ) for i in range(2)]
drpairs += [(drs[0],drs[i] ) for i in range(2,3)]
#drpairs += [(drs[1],drs[3] )]
print(drpairs)

dppairs = [(dps[i],dps[i+1] ) for i in range(2)]
dppairs += [(dps[0],dps[i] ) for i in range(2,3)]
#dppairs += [(dps[0],dps[i] ) for i in [2]]
#dppairs += [(dps[1],dps[3] )]
print(dppairs)

In [ ]:
#pvalues, formatted_pvalues, pair2pv = getPvals(dftmp,fitcol, pairs)
#pair2pv

In [ ]:
#df_all_multi_tsz['error_data_for_calc']

In [ ]:
calc_names_des = ['lh2_ang_valid', 'signed_area2_nn_valid']

In [ ]:
#sns.set(font_scale=1.5)
sns.set_style("whitegrid", {'font_scale':3})

In [ ]:
import seaborn as sns
#%debug
ann = 0
#hue = 'env_nice'
env2color = {'stable':'lightblue', 'random':'orange'}
aspect=1.3
#rng = [2]
#rng = [2,3,4,5,6,7]
rng = [2]
ylim = [-10,10]
#ylim = [-100,100]
#ylim = [-1e10,1e10]
#fsz=13
fd = None
meanfit=1
bnqs = [  ('grid_line_violin_spatial',
           'Spatial distance [deg]',drs,drpairs,
           'dist_rad_from_prevtgt2', 
           f'trial_group_col_calc == "trials" and perturbation_diff_abs == 0'),
        ( 'grid_line_violin_pert',
         'Perturbation difference [deg]',
         dps,dppairs,'perturbation_diff_abs',
         'trial_group_col_calc == "trials" and dist_rad_from_prevtgt2 == 0')]

#for env_type in ['stable','random']:
#    color = env2color[env_type]
color = 'lightblue'
for bn,xlab,order,pairs,fitcol,qs in bnqs:
    #qs0 = f'trial_group_col_calc == "trialwe" and err_sens >= {ylim[0]} and err_sens <= {ylim[1]}'
    #qs = qs + ' and trial_shift_size == 1'
#         if env_type in ['stable', 'random']:
#             qs += ' and env_nice == @env_type'
#             hue = None
    qs_ = qs + ' and err_sens.abs() <= 10 and error_data_for_calc == "signed_area2_nn_valid"'
    dftmp0 = dfes_sw.query(qs_)
    #fitcol = 'dist_rad_from_prevtgt2'
    fg = sns.catplot(kind='violin', data=dftmp0, y='err_sens', 
                      x=fitcol, order = order, #row ='error_data_for_calc',
                     #hue = 'error_data_for_calc',
                      color=color, aspect=aspect,
                    bw=0.15)
    
    for cn in calc_names_des:
        qs__ = qs_+ ' and error_data_for_calc == @cn'
        dftmp = dfes_sw.query(qs__).copy()
        print(cn)
        print(qs__, 'len = ',len(dftmp))
        for alt in ['less','greater','two-sided']:
            print('alt = ',alt)
            pvalues, formatted_pvalues, pair2pv = \
                getPvals(dftmp,fitcol, pairs, alternative=alt)


    for ax in fg.axes.flatten():
        ax.axhline(0,ls=':',c='r')
        ax.set_ylabel('Error sensitivity', fontdict=fd)
        ax.set_xlabel(xlab, fontdict=fd)
        if bn == 'grid_line_violin_spatial':
            ax.set_xticklabels(['same target', 20, 40])
        ax.set_ylim(ylim)
        #ax.set_title(f'{env_type} environment', y = 0.967,
        #            fontdict={'fontsize':15, 'fontweight':'bold'})


    if ann:
        from statannotations.Annotator import Annotator    
        annotator = Annotator(ax,pairs, data=dftmp0, x=fitcol, y='err_sens',
                             plot='violinplot', order=order)
        #annotator.set_custom_annotations(formatted_pvalues)
        annotator.set_pvalues(pvalues)
        annotator.annotate()        

        plotPolys(ax,dftmp,fitcol, degs=rng, mean=meanfit)
        
    ax.set_ylim(*ylim)
    #break
    #bn = 'grid_line_violin_spatial'
    plt.savefig(pjoin(path_fig_nf,f'{bn}.svg'))
    plt.savefig(pjoin(path_fig_nf,f'{bn}.pdf'))

# Working with context similarity

In [ ]:
sns.catplot?

In [ ]:
colnes

In [ ]:
dfes_sw['error_lh_ang_deg'] = dfes_sw['error_lh_ang'] * 180 / np.pi
dfes_sw['error_lh_ang_deg_abs'] = dfes_sw['error_lh_ang_deg'].abs()
dfes_sw['error_area2_signed_nn_abs'] = dfes_sw['error_area2_signed_nn'].abs()
dfes_sw['target_locs_nice'] = np.round(dfes_sw['target_locs']).astype('int') - 90

In [ ]:
len(dfes_sw)

In [ ]:
df_notsame = dfes_sw.query('prev_block_ind_diff > 0')
len(df_notsame)

In [ ]:
calcName2puberr = {'lh_ang':'error_lh_ang_deg'}
#edfc

In [ ]:
dfes_sw['prev_block_ind_diff'] = dfes_sw['prev_block_ind_diff'].astype(int)
hue_order = dfes_sw['prev_block_ind_diff'].unique()
print(hue_order)

In [ ]:
# plot errors
# qs = qs_notspec_not_afterspec
# print(qs)
# df_ = dfcc_all.query(qs)
# np.sum(df_['prev_block_ind_diff'].isna())

from behav_proc import coln_ctx_closeness 

# bw=0.04
#df_notsame = dfes_sw.query('prev_block_ind_diff > 0')
#df_notsame = dfsw_care_clean
#for c, coln_error in zip(['orange','blue'], colnes_abs):
c = 'orange'
coln_error = calcName2puberr[edfc]
for coln in coln_ctx_closeness:
    plt.figure(figsize=(4,3))
    fg = sns.catplot(dfes_sw, y = coln_error, 
        kind='violin', height=3, color=c,
            x = coln,
            cut=coln_error.endswith('_abs'))
    for ax in fg.axes.flatten():
        ax.axhline(0,ls=':',color='red')
        if coln_error.endswith('_abs'):
            ax.set_ylim(0,60)
# hue_order=hue_order,hue=coln, split=True,    
   # break

In [ ]:
dfc_multi_tsz.columns

### ES

In [ ]:
print( dfc_multi_tsz['trial_group_col_calc'].unique() )
print( dfc_multi_tsz['error_data_for_calc'].unique() )
print( dfc_multi_tsz.subject.unique() )

In [ ]:
from behav_proc import coln_ctx_closeness 

c = 'orange'
crop = 5
coln_error = calcName2puberr[edfc]
for coln in coln_ctx_closeness:
    plt.figure(figsize=(4,3))
    fg = sns.catplot(dfes_sw.query('err_sens.abs() < @crop'),
        y = 'err_sens', 
        kind='violin', height=3, color=c,
        x = coln,
        cut=coln_error.endswith('_abs'), scale='width',
        bw = 0.2)
    for ax in fg.axes.flatten():
        ax.axhline(0,ls=':',color='red')        
        ax.set_ylim(-crop,crop)
# hue_order=hue_order,hue=coln, split=True,    

In [ ]:
#qs = ('trial_index > @numtrain and trial_type != "error_clamp"'
#                     ' and special_block_type != "error_clamp_sandwich" and prev_trial_type != "pause"')

In [ ]:
# edfc = 'signed_area2_nn_scaled_ed'
# tl = 'target'
# #tl = 'trial_end'
# #f'subject == "{subj}"' 
# qs_es = ('error_data_for_calc == @edfc'
#           ' and trial_shift_size == 1 '
#           ' and trial_group_col_calc == "trials"'
#           ' and time_locked == @tl'
#          )  

In [ ]:
# TODO put to sameness info to dfcc before calc err sens 
for c, cp in zip(['orange','blue'], coln_pairs):
    
    
    
    
    coln_error = cp[0]
    edfc = cp[-1]
    print(cp)
    dfes = dfc_multi_tsz.query(qs + ' and trial_shift_size == 1 and error_data_for_calc == @edfc')
    dfes = dfes.copy() # otherwise the function below won't work since it modifies inplace
    #setContextSimilarityCols(dfes)
    for coln in coln_ctx_closeness:
        plt.figure(figsize=(4,3))        
        fg = sns.catplot(dfes, y = 'err_sens', 
            col=coln, kind='violin', height=3, color=c)
        for ax in fg.axes.flatten():
            ax.axhline(0,ls=':',color='red')
            ax.set_ylim(-20,20)
        break
    gc.collect()

In [ ]:
plt.close('all')

In [ ]:
numtrain

In [ ]:
qs

In [ ]:
edfc

In [ ]:
for c, cp in zip(['orange','blue'], coln_pairs):
    #coln_error = cp[0]
    edfc = cp[-1]
    dfes = dfc_multi_tsz.query(qs + 
        ' and trial_shift_size == 1 and error_data_for_calc == @edfc')
    fg = sns.catplot(dfes, kind='violin', y='err_sens', height=3)
    for ax in fg.axes.flatten():
        ax.axhline(0,ls=':',color='red')
        ax.set_title(edfc)
        ax.set_ylim(-5,5)

# ES cropping

In [ ]:
edfcs = df_all_multi_tsz['error_data_for_calc'].unique()
print(edfcs)

In [ ]:
cp2dfces_remained_notspec = {}
for cp in coln_pairs[:2][::-1]:

    coln_error = cp[0]
    edfc = cp[-1]

    print(cp)
    tl = 'target'

    qs_es = ('error_data_for_calc == @edfc'
              ' and trial_shift_size == 1 '
              ' and trial_group_col_calc == "trials"'
              ' and time_locked == @tl and trial_index >= @numtrain')

    # qs_es = ('trial_shift_size == 1 '
    #           ' and trial_group_col_calc == "trials"'
    #           ' and time_locked == @tl')


    # if remove after spec, I also kill first trials in block for many
    # qs_notspec_not_afterspec
    qs_es += ' and ' + qs_notspec
    print(qs_es, edfc)

    dfces = dfc_multi_tsz.query(qs_es)
    dfes = df_all_multi_tsz.query(qs_es)

    if 'ang' in coln_error:
        crop_deg = 1
        crop = crop_deg / 180 * np.pi 
        crop_es = 36.452  # we don't want to throw away when correction was small as well
    else:
        # pretty arbitrary
        crop = coln2mvar_std_preverr[coln_error] / 20.
        crop_es = 36
    #print(crop)
    # NOTE: using correction estimated from motor var trials
    # to restrict pert trials is just plain wrong
    crop_corr = coln2mvar_std_corr[coln_error] 
    # true std = 
    print('crop_corr = ',crop_corr)

    qs = '((prev_error.abs() < @crop) and (err_sens.abs() > @crop_es))'
    #qs = '((prev_error.abs() < @crop) or (correction.abs() > @crop_corr))'
    #qs = '((prev_error.abs() < @crop) )'
    dfces_cropped = dfces.query(qs)
    print(crop, len(dfces_cropped), len(dfces_cropped) / len(dfces))

    dfces_remained = dfces.query('not ' + qs)

    print('cropped')
    cols = ['trialwb','err_sens','prev_error', 'correction']
    display(dfces_cropped[cols].describe())
    print('remained', len(dfces_remained))
    display(dfces_remained[cols].describe())

    from behav_proc import qs_notspec_not_afterspec, qs_notspec
    dfces_remained_notspec = dfces_remained.query(qs_notspec)

    print('remained not spec', len(dfces_remained_notspec))
    display(dfces_remained_notspec[cols].describe())
    
    cp2dfces_remained_notspec[cp] = dfces_remained_notspec

In [ ]:
coln_pairs

In [ ]:
coln2mvar_std_preverr

In [ ]:
dfces_cropped.query('subject == "2023-SE1-001"')[['Nctx_app'] + cols_binfo]

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    #print (df)
    display(sz)

In [ ]:
cols = ['subject','Nctx_app','trialwb',
              'perturbation','err_sens']
sz=  dfces_cropped.groupby(cols[:-2]).size()

In [ ]:
sns.pairplot(data=dfces_cropped, 
        vars=cols)

In [ ]:
#display(dfces_cropped['err_sens'].abs().describe())
dfces_remained['err_sens'].hist(bins=100)

In [ ]:
dfcpc_all.query('subject == "2023-SE1-015"')['trial_index'].unique()

In [ ]:
dfces_cropped_smalles = \
    dfces_cropped.query('err_sens.abs() < 1e-4')
print(len(dfces_cropped_smalles))

In [ ]:
from behav_proc import plotAllTajCouples
# these are trials where correction is really small 
# (after looking at plots)
dfces_cropped_smalles = \
    dfces_cropped.query('err_sens.abs() < 1e-3')
plotAllTajCouples(dfces_cropped_smalles, df_all_multi_tsz, dfc_all,
                 params, exitpt_col='time_tmstart2')

In [ ]:
df_ = dfces_cropped.query('trialwb > 0').nlargest(20,'err_sens')
df_[['trial_index','trialwb','err_sens','prev_error','correction']]

In [ ]:
from behav_proc import plotAllTajCouples

plotAllTajCouples(df_, 
      df_all_multi_tsz, dfc_all,
     params, exitpt_col='time_tmstart2')

In [ ]:
#.query('trialwb > 0')
df_ = dfces_remained.nlargest(20,'err_sens')
cols_binfo = ['trial_index','trialwb','prev_trial_type','perturbation','prev_perturbation_valid',
     'err_sens','prev_error','correction']
df_[cols_binfo]

In [ ]:
from behav_proc import plotAllTajCouples

plotAllTajCouples(df_, 
      df_all_multi_tsz, dfc_all,
     params, exitpt_col='time_tmstart2')

In [ ]:
#c = np.isinf( df_all_multi_tsz['err_sens'] ) | np.isna( df_all_multi_tsz['err_sens'] )
#c2 = df_all_multi_tsz['err_sens'] > 0
#df_all_multi_tsz.loc[~c, 'err_sens_log'] 
#NOOOO. log will give negatives itself so there will be bad overlap

In [ ]:
assert not dfes.duplicated().any()

In [ ]:
dfes.error_coln_for_calc

In [ ]:
dfes['Nctx_app']

In [ ]:
'prev_error_area2_signed_nn'

In [ ]:
dfces_remained_notspec.query('err_sens.abs() > 15')[cols_binfo]

In [ ]:
dfces_remained.query('err_sens.abs() > 20')[cols_binfo]

In [ ]:
# this is a bad crop, because it acts on err sens directly instead
# of actin on inv_prev_error
dfces_remained_c2 = dfces_remained.query('err_sens.abs() < 10')
dfces_remained_c2[cols_binfo].describe()

In [ ]:
dfces_remained_c2['prev_error'].abs().describe()

In [ ]:
 0.017461 * 180 / np.pi

In [ ]:
vs = ['block_ind','Nctx_app', 'time_lh', 'ctxid',
      'correction', 'prev_error',
                  'inv_prev_error','error_lh_ang', 
      'err_sens' ]
sns.pairplot(data=dfces_remained, 
             vars=vs
            )
plt.tight_layout()
plt.savefig(pjoin(path_fig_nf, 
        'pairplot_dfces_remained_' + ','.join(vs) + '_' + '.png' ));
plt.close()

In [ ]:
sns.pairplot(data=dfces_remained_notspec, 
             vars=vs
            )
plt.tight_layout()
plt.savefig(pjoin(path_fig_nf, 
        'pairplot_dfces_remained_notspec_' + ','.join(vs) + '_' + '.png' ));
plt.close()

In [ ]:
sns.pairplot(data=dfces_remained_c2, 
             vars=['trial_index', 'trialwb'] + vs
            )
plt.tight_layout()
plt.savefig(pjoin(path_fig_nf, 
        'pairplot_dfces_remained_c2_' + ','.join(vs) + '_' + '.png' ));
plt.close()

In [ ]:
vs = ['block_ind','Nctx_app', 'time_lh',
      'correction', 'prev_error',
                  'error_lh_ang', 'error_lh2_ang',
      'error_area2_signed_nn', 'prev_error_area2_signed_nn']
sns.pairplot(data=dfes, 
             vars=vs
            )
plt.tight_layout()
plt.savefig(pjoin(path_fig_nf, 
        'pairplot_dfes_' + ','.join(vs) + '_' + '.png' ));
plt.close()

In [ ]:
vs = ['block_ind','Nctx_app', 'time_lh',
      'correction', 'prev_error',
                  'inv_prev_error','error_lh_ang', 
      'err_sens' ]
sns.pairplot(data=dfces, 
             vars=vs
            )
plt.tight_layout()
plt.savefig(pjoin(path_fig_nf, 
        'pairplot_dfces_' + ','.join(vs) + '_' + '.png' ));
plt.close()

In [ ]:
fg = sns.catplot(data=dfces_remained_notspec.query('trialwb == 0'),
           kind='swarm', y ='err_sens', x='ctxid')
for ax in fg.axes.flatten():
    ax.axhline(0,ls=':',color='red')
    
plt.suptitle('trialwb = 0')

In [ ]:
dfecp[['subject','block_ind','trial_index','trialwb',coln_error]]

In [ ]:
coln_error

In [ ]:
dfcc_all.columns

In [ ]:
dfecp['trialwb'].describe()

# ES simple violins

In [ ]:
dfces_remained_notspec_joined = []
for cp, df_ in cp2dfces_remained_notspec.items():
    if 'valid' in cp[-1]:
        dfces_remained_notspec_joined += [df_]
dfces_remained_notspec_joined = pd.concat(dfces_remained_notspec_joined)

In [ ]:
from behav_proc import coln2pubname

In [ ]:
#sns.set(font_scale=1.5)
sns.set_style("whitegrid", {'font_scale':1.5})

In [ ]:
palette=['darkgreen','brown']
i = 0
for cp,df_ in list( cp2dfces_remained_notspec.items() ):
    ax = plt.gca()
    df_.hist('err_sens',bins=np.linspace(-3,3,60), ax=ax,
            alpha=0.6,label=coln2pubname[cp[0]] )#, color=palette[i] )
    #plt.xlim(-10,10)
    plt.vlines([0],0,900, color='r', ls='--', lw=4,alpha= 0.7)
    plt.legend()
    plt.suptitle('Error sensitivity')
    plt.title('')
    i += 1
    break
plt.savefig(pjoin(path_fig_nf,'ES_hist_area_only.svg'))
plt.savefig(pjoin(path_fig_nf,'ES_hist_area_only.pdf'))

In [ ]:
coln

In [ ]:
print(len(df_))
fg = sns.catplot(data=dfces_remained_notspec_joined, 
    kind='boxen',  x ='error_data_for_calc',
    y='err_sens' )#, bw=0.0012)
for ax in fg.axes.flatten():
    ax.axhline(0,c='r',ls=':')
    ax.set_ylim(-10,10)

#break

# Compute decaly and ground truth err sens

In [ ]:
dfecp.columns

In [ ]:
colns_err_unpert = [ 'error_unpert_lh2_ang', 'error_unpert_area2_signed_nn']
dfecp = dfcc_all.query('special_block_type == "error_clamp_pair"' 
     ' and prev_perturbation_valid.abs() > 0 ')
rs = []
def f(dftmp, coln):
    assert len(dftmp) == 2
    #coln = 'ang_fb_lh_to_postlh'
    eo0 = dftmp.iloc[0][coln]
    eo1 = dftmp.iloc[1][coln]
    if abs(eo0) < 1e-10:
        print(dftmp.iloc[0])
        return np.nan
    return abs( eo1/eo0 )

for coln in colns_err_unpert:
    print(coln)
    decay0 = dfecp.groupby(['subject','block_ind']).\
        apply(lambda x: f(x, coln))
    decay = decay0.reset_index()
    
    df_ = decay.groupby('subject')[0].median().to_frame()
    
    
    display(decay.describe())
    decay = decay.rename(columns={0:'decay'})
    # bad hack but otherwise it is nonsense
    decay = decay.query('decay < 2')
    display(decay.describe())
    #decay_wsubj = 
    
    
    df_['coln'] = coln
    rs += [df_]
    #desc = decay_wsubj.describe()
    #display (desc )
    
df_ = pd.concat(rs).reset_index().set_index(['subject','coln'])
decay_wsubj = df_.to_dict()
decay_wsubj = decay_wsubj[0]

In [ ]:
decay_wsubj

In [ ]:
cps = list(zip( colns_err_unpert, colnes))
display(cps)

In [ ]:
dfecp = dfcc_all.query('special_block_type == "error_clamp_sandwich"' 
     ' and prev_perturbation_valid.abs() > 0 ')
def f(dftmp, coln_unpert, coln_error):
    assert len(dftmp) == 3
    #coln = 'ang_fb_lh_to_postlh'
    alpha = decay_wsubj[dftmp['subject'].values[0], coln_unpert]
    coln = 'error_unpert_lh2_ang'
    eo0 = dftmp.iloc[0][coln_unpert]
    e1  = dftmp.iloc[1][coln_error]
    eo2 = dftmp.iloc[2][coln_unpert]
    if abs(e1) < 1e-10:
        print(dftmp.iloc[0])
        return np.nan
    return (eo2 - alpha * eo0) /e1

for coln_unpert, coln_error in cps:
    print(coln_unpert, coln_error)

    es_gt0 = dfecp.groupby(['subject','block_ind']).\
        apply(lambda x: f(x, coln_unpert, coln_error))
    es_gt = es_gt0.reset_index()
    es_gt = es_gt.rename(columns={0:'err_sens_gt'})
    display(es_gt)
    desc = es_gt.groupby('subject')['err_sens_gt'].mean().describe()
    display (desc )

In [ ]:
# err / err

In [ ]:
df_all_multi_tsz['is_ctxchange_first_felt']

# Timecourse of ES

In [ ]:
pause_block_inds = dfcpcos.groupby('trial_index').min('time')
pause_block_inds = pause_block_inds['block_ind'].unique()

display(pause_block_inds % 8)

c = df_all_multi_tsz['block_ind'] % 8 == 4
df_all_multi_tsz.loc[c & (df_all_multi_tsz['trialwb'] == 0), 
            ['trial_type','prev_trial_type','err_sens']]

In [ ]:
dfces_remained_notspec['prev_error'].abs().describe()

In [ ]:
# for not so small prev errors and outside spec trials
# err sens gwos really only on the very first trial, then stabilizes

In [ ]:
gc.collect()

In [ ]:
#sns.set(font_scale=1.5)
sns.set_style("whitegrid", {'font_scale':1.5})

In [ ]:
df_ = dfces_remained_notspec_joined.query(
        ' trialwb < 10 and error_data_for_calc == "signed_area2_nn_valid"').copy()
print( df_.groupby('subject').size().mean() )
df_['perturbed'] = df_['perturbation'].abs() >= 1e-10
print(len(df_))
#row='midpause'
#plt.figure(figsize=(5,3))
fg = sns.relplot(data=df_, kind='line', x='trialwb',
                 col = 'error_data_for_calc',
                 y='err_sens', hue='perturbed', palette=['darkgreen','brown'])
for axi,ax in enumerate(fg.axes.flatten()):
    ax.axhline(0,ls=':',color='red')
    
    coln = coln_pairs[1-axi][0]
    ttl = coln2pubname[coln]
    
    ax.set_title(ttl)
    ax.set_ylabel('Error sensitivity')
    ax.set_xlabel('Trial within block')
    ax.set_ylim(-2,1.2)
    
#plt.suptitle('Dynamics of ES')

plt.tight_layout()
#plt.legend(loc='lower right')
#plt.savefig(pjoin(path_fig_nf,'ES_dyn_average_contexts.svg'))
#plt.savefig(pjoin(path_fig_nf,'ES_dyn_average_contexts.pdf'))

plt.savefig(pjoin(path_fig_nf,'ES_dyn_average_contexts_only_area.svg'))
plt.savefig(pjoin(path_fig_nf,'ES_dyn_average_contexts_only_area.pdf'))
#del dfes
#break

In [ ]:
sns.set(font_scale=1.5)

In [ ]:
fg = sns.relplot(data=dfces_remained_notspec_joined, kind='line', x='trialwb',
                 y='err_sens', hue='error_data_for_calc',
    col='perturbation', 
    row='tgti_to_show')#, hue='nonhit')
for ax in fg.axes.flatten():
    ax.axhline(0,ls=':',color='red')
    plt.suptitle(f'previos error')
    ax.set_ylim(-2,2)
    plt.tight_layout()
    
plt.savefig(pjoin(path_fig_nf,'ES_dyn_contexts.svg'))
plt.savefig(pjoin(path_fig_nf,'ES_dyn_contexts.pdf'))

In [ ]:
#for crop in [1,3,5,10]:
for crop in np.linspace(0.017, 0.33, 3):
#for c, cp in zip(['orange','blue'], coln_pairs):
#    coln_error = cp[0]
#    edfc = cp[-1]
    #print(edfs)
    df_ = dfces_remained_notspec_joined.query(
            ' and prev_error.abs() >= @crop and trialwb < 10').copy()
    print( df_.groupby('subject').size().mean() )
    df_['perturbed'] = df_['perturbation'].abs() >= 1e-10
    df_['midpause'] = df_['block_ind'].astype(int) % 8 == 4

    print(len(df_))
    #row='midpause'
    fg = sns.relplot(data=df_, kind='line', x='trialwb',
                     col = 'error_data_for_calc',
                     y='err_sens', hue='perturbed')
    for ax in fg.axes.flatten():
        ax.axhline(0,ls=':',color='red')

    plt.suptitle(f'previos error >= {crop * 180 / np.pi:.3f} deg')
    plt.tight_layout()
    plt.legend(loc='lower right')
    #del dfes
    #break
    gc.collect()

In [ ]:
for crop in np.linspace(0.017, 0.33, 3):
    df_ = dfces_remained_notspec.query(qs_es + \
        ' and prev_error.abs() >= @crop and trialwb < 10')#.copy()
    print( df_.groupby('subject').size().mean() )

    fg = sns.relplot(data=df_, kind='line', x='trialwb',
                     y='err_sens', hue='error_data_for_calc',
        col='perturbation', 
        row='tgti_to_show')#, hue='nonhit')
    for ax in fg.axes.flatten():
        ax.axhline(0,ls=':',color='red')
        plt.suptitle(f'previos error >= {crop * 180 / np.pi:.3f} deg')
        ax.set_ylim(-1,1)
    plt.tight_layout()
    


In [ ]:
for crop in np.linspace(0.017, 0.33, 2):
    df_ = dfces_remained_notspec.query(qs_es + \
        ' and prev_error.abs() >= @crop and trialwb < 10'
                ' and Nctx_app in [1,7]')#.copy()
    print( df_.groupby('subject').size().mean() )

    fg = sns.relplot(data=df_, kind='line', x='trialwb',
                     y='err_sens', hue='Nctx_app',
        col='perturbation', 
        row='tgti_to_show')#, hue='nonhit')
    for ax in fg.axes.flatten():
        ax.axhline(0,ls=':',color='red')
        plt.suptitle(f'previos error >= {crop * 180 / np.pi:.3f} deg')
        ax.set_ylim(-2,2)
    plt.tight_layout()
    


In [ ]:
for crop in np.linspace(0.017, 0.33, 1):
    df_ = dfces_remained_notspec.query(qs_es + \
        ' and prev_error.abs() >= @crop and trialwb < 10'
        ' and trialwb in [1,4]')#.copy()
    print( crop, df_.groupby('subject').size().mean() )

    fg = sns.relplot(data=df_, kind='line', x='Nctx_app',
                     y='err_sens', hue='trialwb',
        col='perturbation', 
        row='tgti_to_show')#, hue='nonhit')
    for ax in fg.axes.flatten():
        ax.axhline(0,ls=':',color='red')
        plt.suptitle(f'previos error >= {crop * 180 / np.pi:.3f} deg')
        ax.set_ylim(-3,3)
    plt.tight_layout()
    


In [ ]:
for crop in np.linspace(0.017, 0.33, 3):
    df_ = dfces_remained_notspec.query(qs_es + \
        ' and prev_error.abs() >= @crop and trialwb == 2')#.copy()
    print( df_.groupby('subject').size().mean() )
    
    fg = sns.relplot(data=df_, kind='line', x='Nctx_app',
                     y='err_sens', hue='error_data_for_calc',
        col='perturbation', 
        row='tgti_to_show')#, hue='nonhit')
    for ax in fg.axes.flatten():
        ax.axhline(0,ls=':',color='red')
        plt.suptitle(f'previos error >= {crop * 180 / np.pi:.3f} deg')
        ax.set_ylim(-1,1)
    plt.tight_layout()

In [ ]:
dfcc_all_['subject'].unique()

In [ ]:
df_startend['subject'].unique()

In [ ]:
df_startend

#### start end

In [ ]:
from behav_proc import getStartEndMultiSubj
dfes_startend = getStartEndMultiSubj(dfes)

In [ ]:
#block_bounds

In [ ]:
df_.query('subject == "2023-SE1-003" and block_ind == 12')

In [ ]:
#df_all_multi_tsz.query('subject == "2023-SE1-003" and block_ind == 12')

In [ ]:
dfcc.query('subject == "2023-SE1-003" and block_ind == 12')

In [ ]:
block_bounds[r == 1]

In [ ]:
r = block_bounds['block_end_trial_index'] - block_bounds['block_start_trial_index']
r.to_frame().describe()

In [ ]:
#dfes.groupby('subject').size()

In [ ]:
# from single imported
for c, cp in zip(['orange','blue'], coln_pairs):
    coln_error = cp[0]
    edfc = cp[-1]
    #print(edfs)
    dfes = dfc_multi_tsz.query(qs + ' and trial_shift_size == 1 and error_data_for_calc == @edfc')
    dfes_startend = getStartEndMultiSubj(dfes)

    for yname in ynames:
        plt.figure()
        ax = sns.violinplot(df_startend, y=yname, x='trial_type', hue='type', split=True)
        ax.axhline(0,ls=':', c='red')
        ax.set_ylim(-2,2)
        ax.set_title(edfc)

In [ ]:
for yname in ynames:
    plt.figure()
    q = 5e-3
    ymax = df_startend[yname].quantile(1-q)
    ymin = df_startend[yname].quantile(q)
    ax = sns.violinplot(df_startend.query('trial_type != "error_clamp"'), 
                   y=yname, x='trial_type', hue='type', split=True)
    ax.set_ylim(ymin,ymax)
    ax.axhline(0,ls=':',c='red')

# Pause related

In [ ]:
colnes

In [ ]:
dfext_ = dfces_remained_notspec_joined.query('trial_index > @numtrain and trial_type != "error_clamp"')
fg = sns.catplot(dfext_, kind='violin', x='prev_trial_type', 
    y='error_area2_signed_nn', 
                 col = 'special_block_type')

#fg = sns.catplot(dfext_, kind='violin', x='prev_trial_type', 
#                 y='error_lh_ang', col = 'special_block_type')


# dfext_ = dfc_multi_tsz.query('trial_index > @numtrain and trial_type != "error_clamp"')
# fg = sns.catplot(dfext_, kind='violin', x='prev_trial_type', 
#                  y='error_area_signed_nn_scaled_ed', col = 'special_block_type')

In [ ]:
pause_inds = dfcpc_all.groupby(['subject','trial_index']).max('time').reset_index()
pause_block_inds = dfcpcos.groupby('trial_index').min('time')
pause_block_inds = pause_block_inds['block_ind'].unique()

display(pause_block_inds % 8)

c = df_all_multi_tsz['block_ind'] % 8 == 4
df_all_multi_tsz.loc[c & (df_all_multi_tsz['trialwb'] == 0), 
            ['trial_type','prev_trial_type','err_sens']]

In [ ]:
pause_inds

In [ ]:
for subj in subjects:
    pause_inds.query('subject == @subj')
    c = dfcpc_all['subject'] == subj
    dfcpc_all.loc[c]

In [ ]:
#pause_inds = dfcpc_all['trial_index'].unique()

pause_inds = np.array(pause_inds)
dfccos['pause_rel'] = -100
dfccos.loc[dfccos['trial_index'].isin(pause_inds-1), 'pause_rel'] = -1
dfccos.loc[dfccos['trial_index'].isin(pause_inds+1), 'pause_rel'] = 1
# for pi in pause_inds:
#     piprev=  pi -1
#     pinext=  pi +1
#     df_ = dfccos.query('trial_index == @piprev')


In [ ]:
df_ = dfcc_all_.query('pause_rel > -10')
#df_ = df_.query('error < 150')

ynames = ['error_area_signed',  'error_distance', 'error']
for yname in ynames:
    plt.figure()
    
#     if yname  == 'error':
#         df_ = df_.query('error < 150')
    sns.violinplot(df_, y=yname, x='pause_rel')